In [ ]:
!pip install Owlready2
!pip install rdflib
!pip install python-Levenshtein
!pip install distance

In [33]:
import os
import tarfile, requests
import rdflib
import difflib
import distance
import distance, Levenshtein
import tarfile, requests

from owlready2 import *
from dateutil import parser
from owlready2 import *
from rdflib import Graph
from sklearn.metrics import precision_recall_fscore_support


# Download data from URI link

In [34]:
if 'iimb_large_30082010.tgz' not in os.listdir():
    !wget http://oaei.ontologymatching.org/2010/im/iimb_large_30082010.tgz
else:
    print("file already exists")
    
print(os.listdir())


file already exists
['iimb_large_30082010.tgz', 'myToolCode.ipynb', '.ipynb_checkpoints', 'IIMB_LARGE']


# Unpack tgz file 

In [35]:
full_path_tar_file=os.path.abspath("iimb_large_30082010.tgz")
tar = tarfile.open(full_path_tar_file)
tar.extractall()
print("directory:\n",os.listdir())


directory:
 ['iimb_large_30082010.tgz', 'myToolCode.ipynb', '.ipynb_checkpoints', 'IIMB_LARGE']


# Print full paths , Load ontologies, parse refalign graphe

In [48]:
# Print full paths
full_path_IIMB_LARGE=os.path.abspath("IIMB_LARGE")
print("IIMB_LARGE $PATH = ",full_path_IIMB_LARGE)
print("iimb_large_30082010.tgz $PATH =",full_path_tar_file)

ref_file_name="refalign.rdf"
directory=os.path.join(full_path_IIMB_LARGE+'/001',ref_file_name)
full_path_refalign=os.path.abspath(directory)
print("refaligne $PATH =",full_path_tar_file)


# Load ontologies
onto = get_ontology(full_path_IIMB_LARGE+"/000/onto.owl").load()
onto2 = get_ontology(full_path_IIMB_LARGE+"/001/onto.owl").load()


# parse refaligne graphe
g_ref = Graph()
g_ref.parse(full_path_refalign, format="xml")


IIMB_LARGE $PATH =  /home/amine/2020_2021_Data_Science/projet_KGD/my_code_for_project/IIMB_LARGE
iimb_large_30082010.tgz $PATH = /home/amine/2020_2021_Data_Science/projet_KGD/my_code_for_project/iimb_large_30082010.tgz
refaligne $PATH = /home/amine/2020_2021_Data_Science/projet_KGD/my_code_for_project/iimb_large_30082010.tgz


<Graph identifier=N5a09e07266164d2a8810b298f2e84fc3 (<class 'rdflib.graph.Graph'>)>

# Exctract functional properties by creating get_func_prop function

In [39]:
def Func(a,b):
    func=a/b  
    return func
    
def get_func_prop(onto,threshold):    
    properties=[]
    str_properties=[]
    num_duplic_properties=[]
    num_uniq_properties=[]

    for i in onto.properties():
        properties.append(i)
    
    buffer1=0
    for i in properties:
        str_properties.append(str(i).split('.')[1])
        buffer1=len(list(i.get_relations()))
        num_duplic_properties.append(buffer1)

    i=0
    j=0
    get_rel=[]
    num_uniq_properties=[]
    func_prop={}
    for n in properties:
        uniq_prop_relations=[]
        get_rel=list(properties[i].get_relations())
        for k in get_rel: 
            if k[0] not in uniq_prop_relations:
                uniq_prop_relations.append(k[0])
        num_uniq_properties.append(len(uniq_prop_relations))
        a=num_uniq_properties[j]
        b=num_duplic_properties[j]
        if b!=0:
            func_prop[n]=Func(a,b)
        j=j+1    
        i=i+1
        
        # Threshold 1 -------------------------
    print("Functionnal properties, After threshold=%d applied: \n "%threshold)
    validated_func_properties=[]
    threshold=1
    for i in func_prop: 
        if func_prop[i]>=threshold:
            validated_func_properties.append(i)
    print("propertiesFunctionality:\n",func_prop)
    print("\nvalidatedFuncPropeties:\n",validated_func_properties,"\n")

    return validated_func_properties



# We create the function compare_prop to compare properties

In [38]:
def compare_prop(entity1,entity2,Property,scoreList):
    
    if Property=="gender":
        ent1=str(getattr(entity1,str(Property))[0]).lower()[0]
        ent2=str(getattr(entity2,str(Property))[0]).lower()[0]
                     
    elif Property=="name":
        ent1=str(entity1.__getattr__(Property)[0]).lower()
        ent2=str(entity2.__getattr__(Property)[0]).lower()
                        
    elif Property=="amount":
        ent1 = str(float(str(getattr(entity1,Property)[0])))
        ent2 = str(float(str(getattr(entity2,Property)[0])))
                    
    else:
        ent1=str(getattr(entity1,str(Property))[0]).lower().replace("-", "/")
        ent2=str(getattr(entity2,str(Property))[0]).lower().replace("-", "/")
                        
    leven = Levenshtein.ratio(ent1, ent2) 
    jac = 1 - distance.jaccard(ent1, ent2)
    sor = 1 - distance.sorensen(ent1, ent2)
    scoreList.append((leven + sor + jac)/3)

    return None

# Invalidation Tool 

In [44]:
# Tool

def tool(g_ref,onto,onto2,func_prop,dataProp,dataTypProperties,objectProperties,conf):
    
    ref_qres = g_ref.query("SELECT ?a ?b WHERE {?c a <http://knowledgeweb.semanticweb.org/heterogeneity/alignment#Cell> ; <http://knowledgeweb.semanticweb.org/heterogeneity/alignment#entity1> ?a; <http://knowledgeweb.semanticweb.org/heterogeneity/alignment#entity2> ?b . }")

    ground_true = []
    measures_predicted = []
    false_detected_rules=[]

    counter=0
    for row in ref_qres:
        ground_true.append(1)
        entity1 = onto.search_one(iri = str(row[0]))
        entity2 = onto2.search_one(iri = str(row[1]))
        
        scoreList=[]
        for p in func_prop:
            Property=str(p).split('.')[1]
            
            if Property in dataTypProperties:
                a=[r[0] for r in p.get_relations()]
                if entity1 in a and entity2 in a:
                    compare_prop(entity1,entity2,Property,scoreList)

            if Property in objectProperties:
                b=[r[0] for r in p.get_relations()]
                if entity1 in b and entity2 in b:
                    propComp1 = getattr(entity1,str(Property))[0]
                    propComp2 = getattr(entity2,str(Property))[0]
                    for ii in dataProp:
                        Prop=str(ii).split('.')[1]
                        cc=[rr[0] for rr in ii.get_relations()]
                        if propComp1 in cc and propComp2 in cc:
                            compare_prop(propComp1,propComp1,Prop,scoreList)
        
        if len(scoreList)!=0:
            if sum(scoreList)/len(scoreList) >= conf:
                measures_predicted.append(1)
            else:
                measures_predicted.append(0)
                false_detected_rules.append((str(row[0]),str(row[1])))
        else:
            measures_predicted.append(0)
            false_detected_rules.append((str(row[0]),str(row[1])))
        

    print(false_detected_rules)
    metrics = precision_recall_fscore_support(ground_true, measures_predicted, average='micro')
    print("---------------------")
    print("Precision: %f"%metrics[0])
    print("Recall: %f"%metrics[1])
    print("F-score: %f"%metrics[2])
    
    return None


# Tests

In [42]:
# Extracting func properties


func_prop=get_func_prop(onto,1)

#-------------------PRINTS----------------------------

print("func_prop:\n")
print("----------\n",func_prop)

objectProperties={}
dataTypProperties={}
dataProp=[]
for prop in onto.object_properties():
        if prop in func_prop:
            name = str(prop).split('.')[1]
            objectProperties[name]=1
print("\nfuncObjProp:\n")
print("--------------\n",objectProperties)

for prop in onto.data_properties():
        if prop in func_prop:
            name = str(prop).split('.')[1]
            dataTypProperties[name]=1
            dataProp.append(prop)
print("\nfuncDataTypeProp:\n")
print("-------------------\n",dataTypProperties)

#------------------------------------------------------

Functionnal properties, After threshold=1 applied: 
 
propertiesFunctionality:
 {IIMBTBOX.amount: 1.0, IIMBTBOX.article: 1.0, IIMBTBOX.calling_code: 1.0, IIMBTBOX.currency: 0.9411764705882353, IIMBTBOX.date_of_birth: 1.0, IIMBTBOX.dialect: 0.13333333333333333, IIMBTBOX.form_of_government: 0.4261168384879725, IIMBTBOX.gender: 1.0, IIMBTBOX.iso_639_1_code: 1.0, IIMBTBOX.name: 1.0, IIMBTBOX.religion: 0.9195402298850575, IIMBTBOX.size: 1.0, IIMBTBOX.acted_by: 0.9953271028037384, IIMBTBOX.born_in: 1.0, IIMBTBOX.capital: 0.9548872180451128, IIMBTBOX.has_capital: 1.0, IIMBTBOX.created_by: 0.8125, IIMBTBOX.creates: 0.34375, IIMBTBOX.directed_by: 0.8849557522123894, IIMBTBOX.directs: 0.911504424778761, IIMBTBOX.estimated_budget_used: 1.0, IIMBTBOX.filmed_in: 0.8547008547008547, IIMBTBOX.mainly_spoken_in: 0.6666666666666666, IIMBTBOX.spoken_in: 0.06976744186046512, IIMBTBOX.native_city_of: 0.6321585903083701, IIMBTBOX.performs: 0.9439252336448598, IIMBTBOX.shot_in: 0.9174311926605505, IIMBTBOX.s

In [47]:
tool(g_ref,onto,onto2,func_prop,dataProp,dataTypProperties,objectProperties,0.65)

[('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k39kc', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item6150292023606147640'), ('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k38l1', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item1871041914381761043'), ('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k3nph', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item1725485724461512562'), ('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k380q', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item2427401387451985825'), ('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k3hsf', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item563854137234860973'), ('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k40f7', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item3334473067559942377'), ('http://oaei.ontologymatching.org/2010/IIMBDATA/m/0k3r4l', 'http://oaei.ontologymatching.org/2010/IIMBDATA/m/item4182069180489509643'), ('http://oaei.ontologymatching.org/2010/I